In [3]:
#from openai import OpenAI
from os import getenv
from dotenv import load_dotenv
#from langchain.chat_models import ChatOpenAI
#from langchain.llms import OpenAI
from langchain_openai import OpenAI


load_dotenv()
#client = OpenAI(api_key=getenv('API_KEY_DALLE'))

#Nombre del despliegue en OpenAI Studio, el modelo es 'text-davinci-003'
#engine = 'Davinci003'
model = 'gpt-3.5-turbo-instruct'
openai_api_version = '2023-12-01'

#Nombre del despliegue para el modelo de embeddings es 'TextEmbeddingAda002'
embeddings_engine = 'TextEmbeddingAda002'

max_tokens = 1000

llm = OpenAI(openai_api_key=getenv('OPEN_API_KEY', ''),
                 model= model,
                 #engine=engine,
                 #openai_api_version=openai_api_version,
                 temperature=0.2)

llm.max_tokens = max_tokens


## 1.1 - ¿Qué es un agente?

El concepto de "agente" es una abstracción a nuestro modelo de lenguaje:

    * Utilizar calculadoras
    * Utilizar calendarios
    * Ejecutar código Python
    * Buscar en Internet
    * etc.

Este tipo deacciones se realizarán mediante herramientas:

    * Proporcionadas por LangChain
    * Diseñadas por nosotros mismos (como veremos más adelante)

Por lo tanto, un agente está compuesto de dos elementos principales:

    * Un LLM base
    * Una o múltiples herramientas con las que interactuar

Dependiendo de la entrada del usuario, el agente puede decidir cuál, si alguna, de estas herramientas llamar. La utilizdad de LangChain se hace evidente en este contexto, ya que proporciona un marco de programación versátil para construir tales sistemas.

## 1.2 - ¿Qué es una herramienta?
En LangChain, las herramientas son funciones que pueden ser llamadas por un agente. Se utilizan para realizar tareas específicas, como acceder a datos, generar texto o traducir idiomas.

Las herramientas se definen como objetos `Tool` que ocntienen tres propiedades:

    * name: El nombre de la herramienta
    * description: Una descripción de la herramienta
    * function: La función a ejecutar cuando se llama a la herramienta

Las herramientas se utilizan para facilitar la creación de agentes complejos que pueden realizar una amplia gama de tareas. P. ej., un agente que ayuda a los usuarios a resolver problemas técnicos podría usar herramientas para acceder a la documentación, buscar soluciones en línea o generar código

A continuación, se presentan algunas herramientas comunes:
    * Wikipedia
    * DuckDuckGoSearch
    * ShellTool

Sin embargo, hay una gran cantidad de ellas en la doc oficial de LangChain (y su nº aumenta cada día).


### Wikipedia

Para búsquedas medianamente estructuradas en Wikipedia

In [4]:
from langchain.tools import WikipediaQueryRun
from langchain.utilities import WikipediaAPIWrapper
from wikipedia import set_lang as wiki_lang

wiki_lang('en')

wikipedia_eng = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(
    doc_content_chars_max = 2000
))

wikipedia_eng.run('Open AI')




'Page: OpenAI\nSummary: OpenAI is a U.S.-based artificial intelligence (AI) research organization founded in December 2015, researching artificial intelligence with the goal of developing "safe and beneficial" artificial general intelligence, which it defines as "highly autonomous systems that outperform humans at most economically valuable work".\nAs one of the leading organizations of the AI spring, it has developed several large language models, advanced image generation models, and previously, released open-source models. Its release of ChatGPT has been credited with starting the AI spring.The organization consists of the non-profit OpenAI, Inc. registered in Delaware and its for-profit subsidiary OpenAI Global, LLC. It was founded by Ilya Sutskever, Greg Brockman, Trevor Blackwell, Vicki Cheung, Andrej Karpathy, Durk Kingma, Jessica Livingston, John Schulman, Pamela Vagata, and Wojciech Zaremba, with Sam Altman and Elon Musk serving as the initial Board of Directors members. Micro

In [5]:
wiki_lang('es')

wiki_es = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(
    lang='es',
    doc_content_chars_max=2000
))

wiki_es.run('Open AI')

'Page: OpenAI\nSummary: OpenAI es una empresa de investigación y despliegue de inteligencia artificial que declara tener como misión asegurar que la inteligencia artificial general beneficie a toda la humanidad.[4]\u200b\nAlgunas de sus áreas de enfoque son los modelos generativos, la seguridad, la robótica y el lenguaje natural. OpenAI también ofrece una plataforma de API que permite a los usuarios acceder y construir sus propios modelos de IA personalizados.\nOpenAI fue fundada en 2015 por Ilya Sutskever, Greg Brockman, Trevor Blackwell, Andrej Karpathy y con Sam Altman y Elon Musk como miembros iniciales de la junta.[5]\u200b[6]\u200b Se fundó principalmente como una organización sin fines de lucro, pero en 2019 se convirtió en una empresa híbrida, con una entidad sin fines de lucro y otra con fines de lucro.\nAlgunos de sus proyectos más destacados son ChatGPT, un chatbot (Modelo de lenguaje) que puede ver, oír y hablar[7]\u200b; DALL·E 3, un modelo de imágenes que puede crear imág

### DuckDuckGoSearch

Para búsquedas "raw" en Internet

`Nota:` En este caso, no he encontrado la manera de indicarle que haga las búsquedas en español, pero puede haber alguna forma. Sino, siempre podemos traducir el resultado como paso intermedio a que nuestro modelo lo procese.

pip install duckduckgo-search

In [6]:
from langchain.tools import DuckDuckGoSearchRun

search = DuckDuckGoSearchRun()

search.run("Open AI")

"Careers at OpenAI. Developing safe and beneficial AI requires people from a wide range of disciplines and backgrounds. I encourage my team to keep learning. Ideas in different topics or fields can often inspire new ideas and broaden the potential solution space. Lilian Weng Safety Systems at OpenAI. OpenAI unveiled Voice Engine, an A.I. technology that can recreate a person's voice from a 15-second recording. Amazon said it had added $2.75 billion to its investment in Anthropic, ... Feb. 16, 2024. OpenAI has completed a deal that values the San Francisco artificial intelligence company at $80 billion or more, nearly tripling its valuation in less than 10 months, according to ... OpenAI is an artificial intelligence research company that produces services such as ChatGPT. Here's what you need to know. Artificial intelligence, or AI, is actively changing how we interact with technology. Generative AI is a type of AI technology that can produce text, images, audio, and more, when prompte

c:\Users\rafacampa9\.virtualenvs\Langchain-NIv1GH6l\Lib\site-packages\curl_cffi\aio.py:205: UserWarning: Curlm alread closed! quitting from process_data
  warnings.warn("Curlm alread closed! quitting from process_data")


### Shell

Permite a los agentes ejecutar comandos de shell. Esto puede ser muy útil para tareas que requieren acceso al SO, como interactuar con el sistema de archivos o ejecutar apps.

`Nota:`
* No funciona en Windows
* Es peligroso dejar el control de nuestro sistema de archivos, lo mejor sería encapsularlo en una máq virtual y prohibirle ciertos comandos (por si acaso)

In [7]:
from langchain.tools import ShellTool

shell_tool = ShellTool()
print(shell_tool.run(
    {
        "commands": [
            "echo 'Hello World'",
            "dir"
        ]
    }
))



Executing command:
 ["echo 'Hello World'", 'dir']
'Hello World';dir



c:\Users\rafacampa9\.virtualenvs\Langchain-NIv1GH6l\Lib\site-packages\langchain_community\tools\shell\tool.py:32: UserWarning: The shell tool has no safeguards by default. Use at your own risk.
  warnings.warn(


## 1.3 - Tipos de agentes

Hay dos tipos de agentes principales:
* Agentes de Acción: deciden una acción a tomar y realizan esa acción paso a paso
* Agentes de Planificación y Ejecución (Experimental): estos agentes primero deciden un plan de acciones a tomar y luego ejecutan esas acciones paso a paso


# 2 - Agentes de Acción

Pseudocódigo de alto nivel de los agentes de acción:
1. Se recibe una entrada del usuario
2. El agente decide qué herramienta utilizar, si es que hay alguna, y cuál debe ser la entrada de esa herramienta
3. A continuación, se llama a esa herramienta con esa entrada de herramienta y se registra una observación (esto es simplemente la salida de llamar a esa herramienta con esa entrada de herramienta).
4. Ese historial, la entrada de herramienta y observación final se devuelve al agente, y éste decide qué paso dar a continuación.
5. Esto se repite hasta que el agente decide que ya no necesita utilizar ninguna herramienta, y entonces responde directamente al usuario.

`4 tipos de agentes de acción (inspirados del paper: "ReAct: Synergizing Reasoning and Acting in Language Models")`:

* ReAct Zero-shot
* ReAct Conversational
* ReAct Docstore
* ReAct Self-ask

AgentTypes en la doc de LangChain.


## 2.1 - ReAct Zero-shot

pip install numexpr

`Nota importante: ` Al interactuar con agentes, es muy importante establecer el parámetro `max_iterations` porque los agentes pueden quedarse atrapados en bucles infinitos que consumen muchos tokens. El valor predeterminado es 15 para permitir el uso de muchas herramientas y razonamientos complejos, pero paara la mayoría de las apps es mejor mantenerlo mucho más bajo

In [9]:
from langchain.agents import AgentType, Tool, load_tools, initialize_agent


tools = load_tools(
    ["llm-math"],
    llm=llm
)


zero_shot_agent_executor = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

c:\Users\rafacampa9\.virtualenvs\Langchain-NIv1GH6l\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [10]:
zero_shot_agent_executor("¿Cuánto es 101 elevado a 9")

c:\Users\rafacampa9\.virtualenvs\Langchain-NIv1GH6l\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
 I should use a calculator to solve this exponent problem
Action: Calculator
Action Input: 101^9
Observation: Answer: 1093685272684360901
Thought: I now know the final answer
Final Answer: 1093685272684360901

> Finished chain.


{'input': '¿Cuánto es 101 elevado a 9', 'output': '1093685272684360901'}

Si nos fijamos, el agente utiliza el inglés como idioma "base". Como hemos visto en otras clases, podemos modificar este comportamiento modificando los prompts de la cadena.

In [11]:
print(zero_shot_agent_executor.agent.llm_chain.prompt.template)

Answer the following questions as best you can. You have access to the following tools:

Calculator: Useful for when you need to answer questions about math.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [Calculator]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


En este caso, vamos a tomar un enfoque diferente y vamos a traducir el texto de español a inglés antes de pasárselo al agente.

In [12]:
from langchain.callbacks import get_openai_callback
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

def call_agent_with_translate(agent, es_eng_chain, eng_es_chain, query):
    print(f'Input: {query}')

    with get_openai_callback() as cb:
        query_translated = es_eng_chain(query)['text']
        query_translated = query_translated.replace('\n', '')
        es_eng_token_count = cb.total_tokens
    print(f'Input (traducido al inglés): {query_translated}')

    with get_openai_callback() as cb:
        result = agent(query_translated)
        agent_token_count = cb.total_tokens
    print(f"Output: {result['output']}")


    with get_openai_callback() as cb:
        output_translated = eng_es_chain(result['output'])['text']
        output_translated = output_translated.replace('\n', '')
        eng_es_token_count = cb.total_tokens
    print(f'Output (traducido al español): {output_translated}')

    total_tokens = es_eng_token_count + agent_token_count + eng_es_token_count

    print(f'He usado un total de {total_tokens} tokens')

    return result

#Chain para traducir el texto de entrada de español a inglés
translate_eng_es_prompt = ChatPromptTemplate.from_template(
        "Translate the following spanish text to english (write only the translated text): {input}"
    )

translate_eng_es_chain = LLMChain(llm=llm, prompt=translate_eng_es_prompt)

    #Chain para traducir el texto de salida del modelo de inglés a español
translate_es_eng_prompt = ChatPromptTemplate.from_template(
        "Si el siguiente texto está en inglés, tradúcelo al español, en caso contrario, simplemente devuelve el ismo texto: {input}"
    )

translate_es_eng_chain = LLMChain(llm=llm, prompt=translate_es_eng_prompt)



In [16]:
result = call_agent_with_translate(
    zero_shot_agent_executor,
    translate_eng_es_chain,
    translate_es_eng_chain,
    "Podrías decirme cuánto es 8 x 21?"
)

Input: Podrías decirme cuánto es 8 x 21?
Input (traducido al inglés): Could you tell me how much is 8 x 21?


> Entering new AgentExecutor chain...
 I should use a calculator to multiply 8 by 21.
Action: Calculator
Action Input: 8 x 21
Observation: Answer: 168
Thought: I now know the final answer.
Final Answer: 168

> Finished chain.
Output: 168
Output (traducido al español): 168
He usado un total de 658 tokens


## 2.2 - ReAct Conversacional

El agente zero-shot es interesante, pero no tiene memoria. ¿Qué pasa si queremos un asistente que recuerde las cosas que hemos hablado y que también pueda razonar sobre ellas y utilizar herramientas? Para eso tenemos el agente ReAct Conversacional

In [17]:
from langchain.memory import ConversationBufferMemory

tools = load_tools(
    ["llm-math"],
    llm=llm
)

memory = ConversationBufferMemory(memory_key = "chat_history")

conversational_agent_executor = initialize_agent(
    agent = AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=3,
    memory=memory
)

result = call_agent_with_translate(
    conversational_agent_executor,
    translate_eng_es_chain,
    translate_es_eng_chain,
    "Podrías decirme cuál es el resultado de 100 + 25?"
)

Input: Podrías decirme cuál es el resultado de 100 + 25?
Input (traducido al inglés): Could you tell me what is the result of 100 + 25?


> Entering new AgentExecutor chain...

Thought: Do I need to use a tool? Yes
Action: Calculator
Action Input: 100 + 25
Observation: Answer: 125
Thought: Do I need to use a tool? No
AI: The result of 100 + 25 is 125.

> Finished chain.
Output: The result of 100 + 25 is 125.
Output (traducido al español): El resultado de 100 + 25 es 125.
He usado un total de 1166 tokens


In [19]:
result = call_agent_with_translate(
    conversational_agent_executor,
    translate_eng_es_chain,
    translate_es_eng_chain,
    "Qué es lo último que te he preguntado en nuestra conversación?"
)

Input: Qué es lo último que te he preguntado en nuestra conversación?
Input (traducido al inglés): What was the last thing I asked you in our conversation?


> Entering new AgentExecutor chain...
AI: The last thing you asked me was "Could you tell me what is the result of 100 + 25?"

> Finished chain.
Output: The last thing you asked me was "Could you tell me what is the result of 100 + 25?"
Output (traducido al español): Lo último que me preguntaste fue "¿Podrías decirme cuál es el resultado de 100 + 25?"
He usado un total de 558 tokens


## 2.3 - ReAct Docstore

Este tipo de agente es similar a los vistos hasta ahora, pero incluye la interacción con un almacén de docs. Solo tendrá dos herramientas a su disposición:
* `Search` (Búsqueda). El agente usa esta herramienta para encontrar un artículo relevante
* `Lookup` (Consulta). El agente intenta encontrar la info correcta del artículo
Es el agente por defecto en el caso que queramos hacer búsquedas simples en wikipedia o almacenes de docs similares.

`Nota`: ReAct Docstore no está pensado para trabajar con memoria por defecto. En tal caso, tendríamos que preparar nuestro propio Agente customizado

In [20]:
from langchain import Wikipedia
from langchain.agents.react.base import DocstoreExplorer
from wikipedia import set_lang as wiki_lang

wiki_lang('en')

docstore = DocstoreExplorer(Wikipedia())
tools = [
    Tool(
        name = "Search",
        func = docstore.search,
        description="search wikipedia"
    ),
    Tool(
        name="Lookup",
        func = docstore.lookup,
        description='lookup a term in wikipedia'
    )
]

docstore_agent_executor = initialize_agent(
    tools,
    llm,
    agent=AgentType.REACT_DOCSTORE,
    verbose=True,
    max_iterations=10
)

c:\Users\rafacampa9\.virtualenvs\Langchain-NIv1GH6l\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain.agents.react.base.DocstoreExplorer` was deprecated in langchain 0.1.0 and will be removed in 0.2.0
  warn_deprecated(


El prompt asociado al modelo difiere de los anteriores en cuanto que contiene varios ejs del bucle Pregunta > Pensamiento > Acción > Observación, que incluyen las herramientas de búsqueda y consulta.

`Nota`: Este bucle se explica con más detalle (al igual que con más ejs en el paper original `ReAct: Synergizing Reasoning and Acting in Language Models`)

In [21]:
print(docstore_agent_executor.agent.llm_chain.prompt.template)



Question: What is the elevation range for the area that the eastern sector of the Colorado orogeny extends into?
Thought: I need to search Colorado orogeny, find the area that the eastern sector of the Colorado orogeny extends into, then find the elevation range of the area.
Action: Search[Colorado orogeny]
Observation: The Colorado orogeny was an episode of mountain building (an orogeny) in Colorado and surrounding areas.
Thought: It does not mention the eastern sector. So I need to look up eastern sector.
Action: Lookup[eastern sector]
Observation: (Result 1 / 1) The eastern sector extends into the High Plains and is called the Central Plains orogeny.
Thought: The eastern sector of Colorado orogeny extends into the High Plains. So I need to search High Plains and find its elevation range.
Action: Search[High Plains]
Observation: High Plains refers to one of two distinct land regions
Thought: I need to instead search High Plains (United States).
Action: Search[High Plains (United St

In [22]:
result = call_agent_with_translate(
    docstore_agent_executor,
    translate_eng_es_chain,
    translate_es_eng_chain,
    "Cuáles han sido las principales investigaciones del filósofo griego Aristóteles?"
)

Input: Cuáles han sido las principales investigaciones del filósofo griego Aristóteles?
Input (traducido al inglés): What have been the main investigations of the Greek philosopher Aristotle?


> Entering new AgentExecutor chain...
Thought: I need to search Aristotle and find his main investigations.
Action: Search[Aristotle]
Observation: Aristotle (384–322 BC) was an Ancient Greek philosopher and polymath. His writings cover a broad range of subjects spanning the natural sciences, philosophy, linguistics, economics, politics, psychology, and the arts. As the founder of the Peripatetic school of philosophy in the Lyceum in Athens, he began the wider Aristotelian tradition that followed, which set the groundwork for the development of modern science.
Little is known about Aristotle's life. He was born in the city of Stagira in northern Greece during the Classical period. His father, Nicomachus, died when Aristotle was a child, and he was brought up by a guardian. At 17 or 18, he joined 

# 2.4 - ReAct Self-ask

Es nuestra primera elección de agente para cuando queremos utilizar un LLM con el objetivo de extraer info con un motor de búsqueda. El agente hará preguntas de seguimiento y utilizará la funcionalidad de búsqueda para obtener respuestas intermedias que lo ayuden a llegar a una respuesta final.

Utilizaremos el motor de búsqueda DuckDuckGo porque es gratuito y no requiere una clave.

`Nota`: Al igual que ReAct Docstore, ReAct Self-ask no está pensado para trabajar con memoria por defecto. En tal caso, tendríamos que preparar nuestro propio Agente customizado

In [23]:
from langchain.tools import DuckDuckGoSearchRun

search = DuckDuckGoSearchRun()
tools = [
    Tool(
        name = "Intermediate Answer",
        func = search.run,
        description="duckduckgo search"
    )
]

self_ask_with_search_agent_executor = initialize_agent(
    tools,
    llm,
    agent=AgentType.SELF_ASK_WITH_SEARCH,
    verbose=True,
    max_iterations=10
)

Como podemos ver a continuación, el prompt del agente es básicamente una serie de ejs para mostrarle al LLM cómo hacer preguntas de seguimiento a una herramienta de búsqueda hasta que pueda llegar a la respuesta final

In [24]:
print(self_ask_with_search_agent_executor.agent.llm_chain.prompt.template)

Question: Who lived longer, Muhammad Ali or Alan Turing?
Are follow up questions needed here: Yes.
Follow up: How old was Muhammad Ali when he died?
Intermediate answer: Muhammad Ali was 74 years old when he died.
Follow up: How old was Alan Turing when he died?
Intermediate answer: Alan Turing was 41 years old when he died.
So the final answer is: Muhammad Ali

Question: When was the founder of craigslist born?
Are follow up questions needed here: Yes.
Follow up: Who was the founder of craigslist?
Intermediate answer: Craigslist was founded by Craig Newmark.
Follow up: When was Craig Newmark born?
Intermediate answer: Craig Newmark was born on December 6, 1952.
So the final answer is: December 6, 1952

Question: Who was the maternal grandfather of George Washington?
Are follow up questions needed here: Yes.
Follow up: Who was the mother of George Washington?
Intermediate answer: The mother of George Washington was Mary Ball Washington.
Follow up: Who was the father of Mary Ball Washin

In [35]:
question = 'Sabes el nombre del futbolista con más balones de oro?'

result = call_agent_with_translate(
    self_ask_with_search_agent_executor,
    translate_eng_es_chain,
    translate_es_eng_chain,
    question
)

Input: Sabes el nombre del futbolista con más balones de oro?
Input (traducido al inglés): Do you know the name of the football player with the most golden balls?


> Entering new AgentExecutor chain...
 Yes.
Follow up: What are golden balls in football?
Intermediate answer: The Ballon d'Or (French pronunciation: [balɔ̃ dɔʁ] ⓘ; lit. 'Golden Ball ') is an annual football award presented by French news magazine France Football since 1956 to honour the player deemed to have performed the best over the previous season. Conceived by sports writers Gabriel Hanot and Jacques Ferran, the Ballon d'Or award was based exclusively on voting by football journalists, from ... The Ballon d'Or, or "Golden Ball" in French, is the most prestigious individual award in world football, recognising the world's finest male and female players. The Golden Ball has been awarded to some of the game's best players, including Diego Maradona, Lionel Messi, Cristiano Ronaldo, Zinedine Zidane, and many others. Footba

c:\Users\rafacampa9\.virtualenvs\Langchain-NIv1GH6l\Lib\site-packages\curl_cffi\aio.py:205: UserWarning: Curlm alread closed! quitting from process_data
  warnings.warn("Curlm alread closed! quitting from process_data")


In [39]:
question = 'Sabes en qué equpos ha jugado Messi?'

result = call_agent_with_translate(
    self_ask_with_search_agent_executor,
    translate_eng_es_chain,
    translate_es_eng_chain,
    question
)

Input: Sabes en qué equpos ha jugado Messi?
Input (traducido al inglés): Do you know which teams Messi has played for?


> Entering new AgentExecutor chain...


ValueError: An output parsing error occurred. In order to pass this error back to the agent and have it try again, pass `handle_parsing_errors=True` to the AgentExecutor. This is the error: Could not parse output:  No, the question is clear and does not require any additional information.

# 3 - Agentes de Planificación y Ejecución

Los agentes `Plan-and-Execute` (PaE) logran un objetivo planificando primero qué hacer y luiego ejecutando las subtareas. Esta idea se inspira en gran medida en `BabyAGI` y luego en el doc `Plan and Solve`

* La planificación casi siempre la realiza un LLM
* La ejecución generalmente la realiza un agente de acción (equipado con herramientas)

Pseudocódigo de alto nivel de los agentes PaE:
* Se recibe alguna entrada del usuario
* El `planificador` enumera los pasos a seguir
* El `ejecutor` recorre la lista de pasos, ejecutándolos

Forma típica de construir un agente PaE:
* `Planificador`: actua a modo de "cerebro" y define qué pasos se deben seguir. Se puede invocar mediante la función `load_chat_planner(llm)`
    - LLM
* `Ejecutor`: decide qué herramientas llamar en cada paso y en qué orden. Se puede invocar con la función `load_agent_executor(llm, tools)`
    - LLM
    - Kit de herramientas

In [40]:
from langchain import LLMMathChain

search = DuckDuckGoSearchRun()
llm_math_chain = LLMMathChain.from_llm(llm=llm, verbose=True)
tools = [
    Tool(
        name='Search',
        func=search.run,
        description="useful for when you need to answer questions about current events"
    ),
    #Esta es la manera "formal" de llamar a 'llm-math'
    Tool(
        name='Calculator',
        func=llm_math_chain.run,
        description='useful for when you need to answer questions about math'
    ),
]

In [41]:
from langchain_experimental.plan_and_execute import PlanAndExecute, load_agent_executor, load_chat_planner

planner = load_chat_planner(llm)
executor = load_agent_executor(llm, tools, verbose=True)
pae_agent_executor = PlanAndExecute(planner=planner, executor=executor)

In [53]:
question = "Calcula el valor de pi elevado a 6. Dime el resultado del cálculo."

result = call_agent_with_translate(
    pae_agent_executor,
    translate_eng_es_chain,
    translate_es_eng_chain,
    question
)

Input: Calcula el valor de pi elevado a 6. Dime el resultado del cálculo.
Input (traducido al inglés): Calculate the value of pi raised to the 6th power. Tell me the result of the calculation.


> Entering new AgentExecutor chain...
Action:
{
  "action": "Final Answer",
  "action_input": "The value of pi is approximately 3.141592653589793."
}



> Finished chain.


> Entering new AgentExecutor chain...
Action:
{
  "action": "Calculator",
  "action_input": "3.141592653589793 ** 6"
}



> Finished chain.


> Entering new AgentExecutor chain...
Action:
{
  "action": "Calculator",
  "action_input": "round(244.140625, 2)"
}



> Finished chain.


> Entering new AgentExecutor chain...
Action:
{
  "action": "Final Answer",
  "action_input": "Rishi Sunak is 43 years old."
}



> Finished chain.
Output: Action:
{
  "action": "Final Answer",
  "action_input": "Rishi Sunak is 43 years old."
}


Output (traducido al español): Acción:{  "acción": "Respuesta final",  "acción_entrada": "Rishi Sunak t